In [15]:
from selenium import webdriver
import datetime
import time
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np

In [16]:
#Open a new web browser.
driver = webdriver.Chrome()

In [ ]:
#Open the web page that we want open and log in.
driver.get('https://shop.wegmans.com/shop/categories/40')

time.sleep(10)

In [ ]:
X_button1 = driver.find_element_by_class_name('icon-delete')
X_button1.click()

In [ ]:
#Grab the webpage as a BeautifulSoup object
soup = BeautifulSoup(driver.page_source,'html.parser')

In [ ]:
#Obtain Product Names and append to blank list
clean_product_names = []
product_name = []
product_names = soup.find_all('div',class_='cell-title-text')

for i in range(len(product_names)):
    each_name = product_names[i].text.strip()
    product_name.append(each_name)
    clean_product_names.append(each_name)

In [ ]:
#Obtain Product Prices and append to blank list (this will accidentally disguise sales prices as regular prices)
clean_prices = []
prices = soup.find_all('div',class_='css-1t08fcf')

for i in range(len(prices)):
    each_price = prices[i].text.strip()
    if 'for' in each_price:
        each_price=each_price[12:17].replace('/','')
    else:
        each_price=each_price[6:12].replace('/','')
        
    clean_prices.append(each_price)

In [ ]:
dirty_prices = []
for i in range(len(prices)):
    dirty_prices.append(prices[i].text.strip())
# to obtain the starting price if it has a sale
# clean_starting_prices = dirty_prices - clean_prices
clean_starting_prices = []

for i in range(len(dirty_prices)):
    clean_starting_price = dirty_prices[i].replace(clean_prices[i],'')
    clean_starting_prices.append(clean_starting_price[1:])

In [ ]:
#Obtain product quantities and append to blank list
clean_quantities = []
quantities = soup.find_all('div',class_='cell-title-text')

for i in range(len(clean_product_names)):
    if "Avg" not in clean_product_names[i]:
        clean_quantities.append('1 Serving')
    else:
        clean_quantities.append(clean_product_names[i].split("Avg.")[1])

In [ ]:
#make a product classification column
product_type = []
product = "Seafood"
for i in range(len(clean_product_names)):
    product_type.append(product)

In [ ]:
#make a column to generate the date the data was pulled
dates = []
todays_date = datetime.date.today()
for i in range(len(clean_product_names)):
    dates.append(todays_date)

In [ ]:
#make a column to generate the store name where the data is from
store = []
for i in range(len(clean_product_names)):
    store.append('Wegmans')

In [ ]:
#Create and outsheet dataframe
Wegmans_Seafood_Data1 = {'Date': dates,'Store':store,'Product Type':product_type, 'Product Name': clean_product_names,'Price':clean_prices,'Quantity Sold In':clean_quantities}
Wegmans_Seafood_Dataframe1 = pd.DataFrame(data=Wegmans_Seafood_Data1)


Wegmans_Seafood_Dataframe1.to_csv ('Wegmans ' +product+' '+str(todays_date)+'.csv', index = False, header=True)

In [70]:
#Close the webpage
driver.close()